In [59]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import ollama

In [58]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="llama3", seed=42)
prompt = ChatPromptTemplate.from_template("Bill Gates worked at {topic}. Change the tag [MASK] to the correct token.")

# using LangChain Expressive Language chain syntax
# learn more about the LCEL on
# /docs/concepts/#langchain-expression-language-lcel
chain = prompt | llm | StrOutputParser()

# for brevity, response is printed in terminal
# You can use LangServe to deploy your application for
# production
print(chain.invoke({"topic": "[MASK]"}))

A classic fill-in-the-blank!

Bill Gates worked at Microsoft.

So, I'll change the tag [MASK] to: **Microsoft**


In [8]:
topic = {"topic": "Space travel"}

async for chunks in chain.astream(topic):
    print(chunks)

Why
 did
 the
 astronaut
 break
 up
 with
 his
 girlfriend
 before
 going
 to
 Mars
?


Because
 he
 needed
 space
!



In [24]:
import json

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

json_schema = {
    "title": "Companie",
    "description": "Identifie the name and change the tag [MASK] by the correct answer.",
    "type": "object",
    "properties": {
        "name": {"title": "Name", "description": "The person's name", "type": "string"},
        "mask": {"title": "Mask", "description": "the word which took place of the tag [MASK]", "type": "string"},
        "fav_food": {
            "title": "CEO of the company",
            "description": "where the person works",
            "type": "string",
        },
    },
    "required": ["name", "mask"],
}

llm = ChatOllama(model="llama3")

messages = [
    HumanMessage(
        content="Please tell me about a person using the following JSON schema:"
    ),
    HumanMessage(content="{dumps}"),
    HumanMessage(
        content="Now, considering the schema, tell me where Bill Gates works for?"
    ),
]

prompt = ChatPromptTemplate.from_messages(messages)
dumps = json.dumps(json_schema, indent=2)

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"dumps": dumps}))

I apologize, but I don't see any information in the provided JSON schema about Bill Gates. The schema appears to be empty. If you meant to provide a different piece of data or a specific person's details, please feel free to share it, and I'll do my best to help!


In [25]:
modelfile='''
FROM llama3
SYSTEM You only answer with json schema.
'''

ollama.create(model='json_llama3', modelfile=modelfile)

{'status': 'success'}

In [32]:
modelfile='''
FROM llama3
SYSTEM You only predict the correct token for the [MASK] tag and provide the 10 most suitable words from highest to lowest probability.
'''

ollama.create(model='mask_llama3-top10', modelfile=modelfile)

{'status': 'success'}

In [35]:
modelfile='''
FROM llama3
SYSTEM You are mario from super mario bros.
'''

ollama.create(model='example', modelfile=modelfile)

{'status': 'success'}

In [34]:
response = ollama.chat(model='mask_llama3', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
print(response['message']['content'])

Apple


In [79]:
modelfile='''
FROM gemma
SYSTEM You only predict the correct token for the [MASK] tag and provide the 10 most suitable words, from the highest to the lowest probability, only with token and probability. The answer should be in dictionary format with the following form list[{'token': token, 'prob': probability}, {'token': token, 'prob': probability},{'token': token, 'prob': probability}, {'token': token, 'prob': probability},{'token': token, 'prob': probability}, {'token': token, 'prob': probability},{'token': token, 'prob': probability}, {'token': token, 'prob': probability},{'token': token, 'prob': probability}, {'token': token, 'prob': probability}]
'''

ollama.create(model='mask_gemma-top10_v1', modelfile=modelfile)

{'status': 'success'}

In [42]:

response = ollama.chat(model='mask_llama3-top10', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
print(response['message']['content'])

Here are the top 10 predictions for the [MASK] token, along with their corresponding probabilities:

1. Apple (0.95)
2. NeXT (0.03)
3. Pixar (0.01)
4. Microsoft (0.005)
5. Google (0.003)
6. HP (0.002)
7. IBM (0.001)
8. Oracle (0.0005)
9. Intel (0.0002)
10. Dell (0.0001)

Note that the probabilities are approximate and based on my training data.


In [64]:
response = ollama.chat(model='mask_llama3-top10_v2', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
print(response['message']['content'])

Here are the top 10 predictions for the [MASK] token:

1. Apple, prob: 0.95
2. Pixar, prob: 0.04
3. NeXT, prob: 0.01

Note that Steve Jobs co-founded Apple and later returned to the company after a hiatus during which he founded NeXT and acquired Pixar (which was then an animation studio).


In [65]:
response = ollama.chat(model='mask_llama3-top10_v3', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
print(response['message']['content'])

Here are the top 10 predictions for the [MASK] token:

token | prob
-----|-----
Apple | 0.92
NeXT | 0.05
Pixar | 0.02
Google | 0.01
Microsoft | 0.01
Amazon | 0.001
Facebook | 0.001
Twitter | 0.0005
Yahoo | 0.0003
Sony | 0.0001

The most likely answer is that Steve Jobs works for Apple, which is correct!


In [70]:
response = ollama.chat(model='mask_llama3-top10_v4', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
print(response['message']['content'])

Here are the top 10 predictions for the [MASK] tag:

1. APPLE (0.935)
2. Apple (0.024)
3. Microsoft (0.003)
4. Google (0.002)
5. IBM (0.001)
6. HP (0.001)
7. Dell (0.000)
8. Oracle (0.000)
9. Intel (0.000)
10. AMD (0.000)

Note that the probabilities are approximate and may vary depending on the specific model used.


In [74]:
response = ollama.chat(model='mask_llama3-top10_v5', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
print(response['message']['content'])

Here are the top 10 predicted tokens along with their probabilities:

{'Apple', '0.9999'}
{'Microsoft', '0.0001'}
{'Google', '0.00005'}
{'IBM', '0.00004'}
{'Oracle', '0.00003'}
{'HP', '0.00002'}
{'Intel', '0.00001'}
{'Cisco', '0.000005'}
{'SAP', '0.000004'}
{'Dell', '0.000003'}

Note: The probabilities are approximate and may vary depending on the model and training data used.


In [83]:
response = ollama.chat(model='mask_llama3-top10_v6', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
answer = response['message']['content']
print(answer)
print(type(answer))

[{'token': 'Apple', 'prob': 0.934}, {'token': 'NeXT', 'prob': 0.0334}, {'token': 'Microsoft', 'prob': 0.0119}, {'token': 'Google', 'prob': 0.0083}, {'token': 'IBM', 'prob': 0.0056}, {'token': 'Oracle', 'prob': 0.0038}, {'token': 'Amazon', 'prob': 0.0024}, {'token': 'Facebook', 'prob': 0.0019}, {'token': 'HP', 'prob': 0.0011}, {'token': 'Intel', 'prob': 0.0007}]
<class 'str'>


In [84]:
response = ollama.chat(model='mask_gemma-top10_v1', messages=[
  {
    'role': 'user',
    'content': 'Steve Jobs works for [MASK]',
  },
])
answer = response['message']['content']
print(answer)
print(type(answer))

[{'token': 'Apple', 'prob': 0.998}, {'token': 'Google', 'prob': 0.001}, {'token': 'Microsoft', 'prob': 0.0009}, {'token': 'Amazon', 'prob': 0.0008}, {'token': 'Facebook', 'prob': 0.0007}, {'token': 'Twitter', 'prob': 0.0006}, {'token': 'Samsung', 'prob': 0.0005}, {'token': 'Sony', 'prob': 0.0004}, {'token': 'Huawei', 'prob': 0.0003}, {'token': 'Intel', 'prob': 0.0002}]
<class 'str'>
